In [2]:
import torch
import torch.nn as nn

In [7]:
GPT_CONFIG_124M = {
    "vocab_size" : 50257,
    "context_length": 1024, 
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

In [14]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Sequential(*[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias = False)


    def forward(self, in_idx):
        batch_size, seq_len, = in_idx.shape
        tok_embs = self.tok_emb(in_idx)
        pos_embs = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embs + pos_embs
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits 


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, cfg):
        super().__init__()

    def forward(self, x):
        return x




In [15]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
batch = []
text1 = "hello my name is"
text2 = "every day is a"

batch.append(torch.tensor(tokenizer.encode(text1)))
batch.append(torch.tensor(tokenizer.encode(text2)))
batch = torch.stack(batch, dim=0)
print(batch)


tensor([[31373,   616,  1438,   318],
        [16833,  1110,   318,   257]])


In [19]:
torch.manual_seed(123)
GPT = DummyGPTModel(GPT_CONFIG_124M)
logits = GPT(batch)
#print(logits)
print(tokenizer.decode(logits[0]))
#print(logits.shape)

TypeError: argument 'tokens': 'Tensor' object cannot be converted to 'Sequence'